# 시험문제
- adult 테스트셋 분류 정확도 95% 의 모델을 찾으세요
- 인구조사 대상의 연간수입이 5만 이상인지를 분류하는 문제
- training 데이터셋으로만 모델을 학습시켜야 함.

In [1]:
import numpy as np
import pandas as pd
from sklearn.tree import export_graphviz

In [5]:
dt = pd.read_csv('adult/adult.data',header=None)
dt.shape

(32561, 15)

In [19]:
test = pd.read_csv('adult/adult.test',skiprows=1,header=None)
test.shape

(16281, 15)

In [20]:
dt.head(3)

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [21]:
test.head()

,0,1,2,3,4,5,6,7,8,9,10,11,12,13,14
0,25,Private,226802,11th,7,Never-married,Machine-op-inspct,Own-child,Black,Male,0,0,40,United-States,<=50K.
1,38,Private,89814,HS-grad,9,Married-civ-spouse,Farming-fishing,Husband,White,Male,0,0,50,United-States,<=50K.
2,28,Local-gov,336951,Assoc-acdm,12,Married-civ-spouse,Protective-serv,Husband,White,Male,0,0,40,United-States,>50K.
3,44,Private,160323,Some-college,10,Married-civ-spouse,Machine-op-inspct,Husband,Black,Male,7688,0,40,United-States,>50K.
4,18,?,103497,Some-college,10,Never-married,?,Own-child,White,Female,0,0,30,United-States,<=50K.


In [22]:
dt.iloc[:, -1].unique()

array([' <=50K', ' >50K'], dtype=object)

In [24]:
test.iloc[:, -1].unique()

array([' <=50K.', ' >50K.'], dtype=object)

In [25]:
df = pd.concat((dt,test), axis=0)

In [26]:
df.shape

(48842, 15)

In [27]:
df = df.replace(' <=50K.',' <=50K', regex=True)
df = df.replace(' >50K.',' >50K', regex=True)

In [35]:
df.columns = ['age', 'workclass', 'fnlwgt', 'education', 'education-num', 'marital-status', 'occupation', 'relationship', 'race', 'sex','capital-gain','capital-loss','hours-per-week','native-country', 'income']

In [81]:
df.head(3)

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,native-country,income
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K


In [36]:
df_hot = pd.get_dummies(df)

In [37]:
df_hot.head(3)

,age,fnlwgt,education-num,capital-gain,capital-loss,hours-per-week,workclass_ ?,workclass_ Federal-gov,workclass_ Local-gov,workclass_ Never-worked,...,native-country_ Scotland,native-country_ South,native-country_ Taiwan,native-country_ Thailand,native-country_ Trinadad&Tobago,native-country_ United-States,native-country_ Vietnam,native-country_ Yugoslavia,income_ <=50K,income_ >50K
0,39,77516,13,2174,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
1,50,83311,13,0,0,13,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0
2,38,215646,9,0,0,40,0,0,0,0,...,0,0,0,0,0,1,0,0,1,0


In [39]:
df_hot.columns

Index(['age', 'fnlwgt', 'education-num', 'capital-gain', 'capital-loss',
       'hours-per-week', 'workclass_ ?', 'workclass_ Federal-gov',
       'workclass_ Local-gov', 'workclass_ Never-worked',
       ...
       'native-country_ Scotland', 'native-country_ South',
       'native-country_ Taiwan', 'native-country_ Thailand',
       'native-country_ Trinadad&Tobago', 'native-country_ United-States',
       'native-country_ Vietnam', 'native-country_ Yugoslavia',
       'income_ <=50K', 'income_ >50K'],
      dtype='object', length=110)

In [69]:
y = df_hot['income_ <=50K']
X = df_hot.iloc[:, :-2]
X.shape, y.shape

((48842, 108), (48842,))

In [70]:
m_train = len(dt)
X_train = X[:m_train]
X_test = X[m_train:]
y_train = y[:m_train]
y_test = y[m_train:]

In [71]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8520929946868954, 0.8519746944290891)

In [72]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)  
rf.score(X_train,y_train), rf.score(X_test, y_test)   # n_estimators=10 => 0.965034965034965   , n_estimators=100 => 실행할 때마다 변경됨.

(0.9999692884125181, 0.8520975370063264)

In [73]:
#p 122
from sklearn.ensemble import GradientBoostingClassifier
gbc = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
gbc.score(X_train,y_train), gbc.score(X_test,y_test) # 과적합

(0.8690457909769356, 0.8712609790553406)

In [74]:
# p132
from sklearn.ensemble import AdaBoostClassifier
adc = AdaBoostClassifier(random_state=0).fit(X_train,y_train)
adc.score(X_train,y_train), adc.score(X_test,y_test) 

(0.8612450477565186, 0.8602665683926048)

In [75]:
# 정규화 [0, 1]
X_norm = (X - np.min(X, axis=0))/(np.max(X, axis=0) - np.min(X, axis=0))

In [76]:
m_train = len(dt)
X_train = X_norm[:m_train]
X_test = X_norm[m_train:]
y_train = y[:m_train]
y_test = y[m_train:]

In [77]:
from sklearn.tree import DecisionTreeClassifier
model = DecisionTreeClassifier(max_depth=5).fit(X_train, y_train)
model.score(X_train, y_train), model.score(X_test, y_test)

(0.8520929946868954, 0.8520361157177078)

In [78]:
from sklearn.ensemble import RandomForestClassifier
rf = RandomForestClassifier(n_estimators=100).fit(X_train, y_train)  
rf.score(X_train,y_train), rf.score(X_test, y_test)   # n_estimators=10 => 0.965034965034965   , n_estimators=100 => 실행할 때마다 변경됨.

(0.9999385768250361, 0.849640685461581)

In [79]:
#p 122
from sklearn.ensemble import GradientBoostingClassifier, AdaBoostClassifier
gbc = GradientBoostingClassifier(random_state=0).fit(X_train, y_train)
gbc.score(X_train,y_train), gbc.score(X_test,y_test) # 과적합

(0.8690457909769356, 0.8712609790553406)

In [80]:
# p132
from sklearn.ensemble import AdaBoostClassifier
adc = AdaBoostClassifier(random_state=0).fit(X_train,y_train)
adc.score(X_train,y_train), adc.score(X_test,y_test) 

(0.8612450477565186, 0.8602665683926048)